# Custom pipeline

In *prose*, `Block` objects (which are dedicated to image analysis and/or modification) can be aranged into `Unit` which have the following workflow:


<div style="text-align:center">
    <img src="../_static/unit_structure.png" width="650px" align=center></img>
</div>


By providing [many pre-implemented blocks](../../guide/api/blocks.rst) one can arange a complete pipeline to achieve a specific analysis. `Block` can also be implemented to include custom processing.

## Objective

In this tutorial we will create a custom`Block` and include it in a pipeline (`Unit`) which goal is to detect and trace a moving object in the following observation:

<div style="text-align:center">
    <img src="../_static/moving_dataset_movie.gif" width="200px" align=center></img>
</div>

Dataset
-------
For this tutorial we will work on a synthetic observation which gif can be seen above. To generate this dataset:


In [ ]:
from prose.datasets import generate_prose_reduction_dataset

dataset = "moving_object_dataset"
generate_prose_reduction_dataset(dataset, moving=[5, [0,40], [75, 60]], n_images=15)

We directly instantiate a `FitsManager` object to explore this folder

In [ ]:
from prose import FitsManager
#
fm = FitsManager(dataset)

In [ ]:
# hidden

from prose import Unit
from prose.blocks import Video

u = Unit([
    Video("moving_dataset_movie.gif", factor=2, from_fits=True),
], fm.images)

_ = u.run()

## Principle

We will use difference imaging to locate the moving object. This is shown in the next code where we can see that difference between consecutive images allow to locate a moving source.

In [ ]:
fm.images

In [ ]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

# loading first and second images
files = fm.images
im0 = fits.getdata(files[5])
im1 = fits.getdata(files[6])

# difference image
im_diff = im1 - im0

# max difference position
position = np.array(np.unravel_index(np.argmax(im_diff), im_diff.shape))[::-1]

plt.imshow(im_diff)
_ = plt.plot(*position, "x", c="k")

## Creating a new :`Block`

Now we can create a `Block` (subclass) that will extract and store this information. As a reminder, a block has the following strucure:

<div style="text-align:center">
    <img src="../_static/block_structure.png" width="400px" align=center></img>
</div>

In [ ]:
from prose import Block
from prose.utils import z_scale

class MovingTarget(Block):
    
    # nothing to initialize
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        # stores last image and positions
        self.last_image = None 
        self.positions = []
        
    def run(self, image):
        
        # difference imaging we did above
        if self.last_image is None:
            self.positions.append(None)
        else:
            im_diff = image.data - self.last_image
            position = np.unravel_index(np.argmax(im_diff), im_diff.shape)[::-1]
            self.positions.append(position)
         
        self.last_image = image.data

    def terminate(self):
        
        # We finish by plotting positions over the last image
        plt.imshow(z_scale(self.last_image), cmap="Greys_r")
        coords_path = np.array(self.positions[1::])
        plt.plot(*coords_path.T, "-o", ms=4)

We can now place our new block in a `Unit` and run it:

In [ ]:
from prose import Unit

pipeline = Unit([
    MovingTarget()
], fm.images)

_ = pipeline.run()

In [ ]:
# hidden
from shutil import rmtree

rmtree(dataset)